In [1]:
import os
import cv2
import glob
import pickle
import requests
import numpy as np
from io import BytesIO
from matplotlib import cm
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageDraw, ImageEnhance

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.backend import resize_images
from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.resnet50 import ResNet50
# from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
# from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.nasnet import preprocess_input, decode_predictions, NASNetLarge

C:\Users\zkalyuzhner\.conda\envs\neurot\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zkalyuzhner\.conda\envs\neurot\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zkalyuzhner\.conda\envs\neurot\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\zkalyuzhner\.conda\envs\neurot\lib\site-packages\ten

In [7]:
with open("kay_data/kay_brain_responses_test_roi_list.pickle", "rb") as input_file:
    kay_brain_responses_test_roi_list = pickle.load(input_file)

In [2]:
with open("kay_data/stimuli_test_features.pickle", "rb") as input_file:
    stimuli_test_features = pickle.load(input_file)

In [3]:
with open("kay_data/all_voxels_test_responses.pickle", "rb") as fp:
    p_test = pickle.load(fp)

In [4]:
with open("kay_data/SparseLinearRegression_all_voxels.pickle", "rb") as fp:
    slr_model = pickle.load(fp)

In [31]:
with open("kay_data/LinearRegression_all_voxels.pickle", "rb") as fp:
    lr_model = pickle.load(fp)

In [5]:
model = NASNetLarge(weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.summary()

In [ ]:
model.layers[-2], model.get_layer('global_average_pooling2d')

In [ ]:
model.layers[-2:]

In [8]:
m = Sequential()
for layer in model.layers[-1:]:
    m.add(layer)

In [ ]:
intermediate_output = m.predict(np.expand_dims(image_features, axis=0))

In [ ]:
intermediate_output.shape
p = decode_predictions(intermediate_output, top=1)[0]

In [ ]:
p

In [39]:
predicted_features_vectors.shape

(1, 4032)

In [48]:
x_test = list()
y_test = list()

for i, res_roi in tqdm_notebook(enumerate(kay_brain_responses_test_roi_list)):
#     if i > 100: break
    img_name = 'stimuli_test_{0}_color_superes'.format(i)
    stimuli = stimuli_test_features[img_name]
    imagenet_label = stimuli[0][0]
    imagenet_class = stimuli[0][1]
    imagenet_score = stimuli[0][2]
    image_features = stimuli[1]
    
    res = res_roi[0]
    roi = res_roi[1]
    
#     predicted_features_vectors = slr_model.predict(np.array([res]))
    predicted_features_vectors = lr_model.predict([res])
    
#     intermediate_output = m.predict(np.expand_dims(predicted_features_vectors, axis=0))
    intermediate_output = m.predict(predicted_features_vectors)
    predicted_classes = decode_predictions(intermediate_output, top=1)[0]
    print('{}\nImageNet class: {}\npredicted class: {}'.format(img_name, imagenet_class, predicted_classes))
    
#     print(img_name, predicted_classes, imagenet_label, imagenet_class)
    print()
    
#     x_test.append(res)
#     y_test.append(image_features)

# x_test = np.array(x_test)
# y_test = np.array(y_test)
# x_test.shape, y_test.shape

C:\Users\zkalyuzhner\.conda\envs\neurot\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


stimuli_test_0_color_superes
ImageNet class: steel_arch_bridge
predicted class: [('n03126707', 'crane', 1.0)]

stimuli_test_1_color_superes
ImageNet class: trilobite
predicted class: [('n04192698', 'shield', 1.0)]

stimuli_test_2_color_superes
ImageNet class: spoonbill
predicted class: [('n04192698', 'shield', 1.0)]

stimuli_test_3_color_superes
ImageNet class: swab
predicted class: [('n02950826', 'cannon', 1.0)]

stimuli_test_4_color_superes
ImageNet class: bell_pepper
predicted class: [('n04192698', 'shield', 1.0)]

stimuli_test_5_color_superes
ImageNet class: radio_telescope
predicted class: [('n04192698', 'shield', 1.0)]

stimuli_test_6_color_superes
ImageNet class: trench_coat
predicted class: [('n02916936', 'bulletproof_vest', 1.0)]

stimuli_test_7_color_superes
ImageNet class: volcano
predicted class: [('n04192698', 'shield', 1.0)]

stimuli_test_8_color_superes
ImageNet class: cornet
predicted class: [('n02916936', 'bulletproof_vest', 1.0)]

stimuli_test_9_color_superes
ImageNet

predicted class: [('n03216828', 'dock', 1.0)]

stimuli_test_81_color_superes
ImageNet class: bighorn
predicted class: [('n04005630', 'prison', 1.0)]

stimuli_test_82_color_superes
ImageNet class: damselfly
predicted class: [('n02916936', 'bulletproof_vest', 1.0)]

stimuli_test_83_color_superes
ImageNet class: bell_cote
predicted class: [('n04192698', 'shield', 1.0)]

stimuli_test_84_color_superes
ImageNet class: leopard
predicted class: [('n03216828', 'dock', 1.0)]

stimuli_test_85_color_superes
ImageNet class: dogsled
predicted class: [('n09332890', 'lakeside', 1.0)]

stimuli_test_86_color_superes
ImageNet class: street_sign
predicted class: [('n03126707', 'crane', 1.0)]

stimuli_test_87_color_superes
ImageNet class: military_uniform
predicted class: [('n03126707', 'crane', 1.0)]

stimuli_test_88_color_superes
ImageNet class: Arabian_camel
predicted class: [('n09332890', 'lakeside', 1.0)]

stimuli_test_89_color_superes
ImageNet class: acorn_squash
predicted class: [('n09332890', 'lake

In [18]:
img_name, predicted_classes, imagenet_label, imagenet_class

('stimuli_test_0_color_superes',
 [('n03126707', 'crane', 1.0),
  ('n15075141', 'toilet_tissue', 0.0),
  ('n02396427', 'wild_boar', 0.0)],
 'n04311004',
 'steel_arch_bridge')